In [13]:
import sklearn
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import json
import re
import uuid
import os
import time

from goose3 import Goose
from collections import Counter

headlineRegex = re.compile(r'http.*?/world')

keyWords = [""]
#keyWords = ["talk", "show"]
#keyWords = ["measles", "disease"]
#domain = "nationalreview.com"
#domain = "dailywire.com"
#domain = "foxnews.com"
#domain = "bbc.com"
#domain = "economist.com"
#domain = "npr.org"
#domain = "slate.com"

#    "huffpost.com",
#    "nytimes.com",
#    "washingtonpost.com",
#    "nationalreview.com",
#   "dailywire.com",
#    "foxnews.com",
#    "bbc.com",
#    "cnn.com",
#    "economist.com",
#    "npr.org",
#    "slate.com"

domains = [
    "nationalreview.com",
    "foxnews.com",
    "cnn.com",
    "npr.org"
]

domains = [
    "nature.com",
]

themes = ["EDUCATION",
"ELECTION",
"SCIENCE",
"DEMOCRACY",
"REFUGEES",
"EXTREMISM",
"TAX_FNCACT_PRESIDENTS",
"UNGP_HEALTHCARE",
"CORRUPTION",
"DRUG_TRADE",
"AGRICULTURE",
"ECON_STOCKMARKET",
"BULLYING",
"FREESPEECH",
"SOC_INNOVATION",
"HEALTH_VACCINATION",
"MOVEMENT_ENVIRONMENTAL",
"MOVEMENT_SOCIAL",
"NATURAL_DISASTER_FLOODS",
"NATURAL_DISASTER_HURRICANES",
"PROPAGANDA",
"ECON_ENTREPRENEURSHIP",
"PUBLIC_TRANSPORT",
"IDEOLOGY",
"DISCRIMINATION",
"IMMIGRATION",
"POVERTY",
"UNEMPLOYMENT",
"CYBER_ATTACK",
"SOVEREIGNTY",
"TAX_DISEASE_DISEASES",
"INSURGENCY",
"PERSECUTION",
"MEDIA_CENSORSHIP",
"TAX_FNCACT_VICE_PRESIDENT",
"MANMADE_DISASTER_PLANE_CRASH"]

themes = [""]

baseDir = "/tmp"
baseUri = "https://api.gdeltproject.org/api/v2/doc/doc?query="
attempts = 3
maxrecords = 250
if keyWords:
    queryBase = "%20".join(keyWords)

g = Goose()

#    "STARTDATETIME" : "20190101000000",
#    "ENDDATETIME": "20190606000000",
 
#    "STARTDATETIME" : "20180601000000",
#    "ENDDATETIME": "20181231000000",

#    "STARTDATETIME" : "20180101000000",
#    "ENDDATETIME": "20180601000000",

#    "STARTDATETIME" : "20170101000000",
#    "ENDDATETIME": "20170601000000",

#    "STARTDATETIME" : "20170601000000",
#    "ENDDATETIME": "20171231000000",

#    "STARTDATETIME" : "20160601000000",
#    "ENDDATETIME": "20161231000000",

#    "STARTDATETIME" : "20160101000000",
#    "ENDDATETIME": "20160601000000",
#"timespan" : "24months",
uriBuilder = {
    "mode" : "artlist",
    "maxrecords" : str(maxrecords),
    "STARTDATETIME" : "20161001000000",
    "ENDDATETIME": "20161231000000",
    "format" : "json"
}

for newsDomain in domains:
    for newsTheme in themes:
    #    time.sleep(1)
    #    queryBuilder = {
    #        "sourcelang" : "english",
    #        "domainis": newsDomain,
    #        "theme": newsTheme
    #    }
        time.sleep(1)
        queryBuilder = {
            "sourcelang" : "english",
            "domainis": newsDomain
        }
        
        query = queryBase
        for key, val in queryBuilder.items():
            query += "%20" + key + ":" + val

        targetUri = baseUri + query
        for key, val in uriBuilder.items():
            targetUri += "&" + key + "=" + val

        attempt = 1
        while attempt <= attempts:
            try:
                print(targetUri)
                with urllib.request.urlopen(targetUri) as url:
                    cleanQuery = url.read().replace(b'\n', b'')
                    string = cleanQuery.decode('utf-8')

                break
            except (ConnectionError, HTTPError, Timeout):
                attempt += 1
                print('need to retry query')
                time.sleep(5)

        #print('need to retry query')
        #continue
        #with urllib.request.urlopen(targetUri) as url:
        #    cleanQuery = url.read().replace(b'\n', b'')
        #    string = cleanQuery.decode('utf-8')
            
        #print("query: %s" % string)
        print("data for %s %s" % (newsDomain, newsTheme))
        if string == '{}':
            #print(targetUri)
            print("no data for %s %s" % (newsDomain, newsTheme))
            continue

        artDict = json.loads(string)
        articles = artDict.get("articles")

        urls = [val for item in articles for key, val in item.items() if key=="url"]
        headlines = [val for item in articles for key, val in item.items() if key=="title"]
        seenDates = [val for item in articles for key, val in item.items() if key=="seendate"]
        
        time.sleep(1) 
        for newsUrl, newsHeadline, newsDate in zip(urls, headlines, seenDates):
            try:
                article = g.extract(url=newsUrl)
                fileData = {
                    "domain": newsDomain,
                    "theme": newsTheme,
                    "headline": newsHeadline,
                    "date": newsDate,
                    "url": newsUrl,
                    "title": article.title,
                    "article": article.cleaned_text
                }
        
                filename = newsDomain + "." + newsTheme + "." + str(uuid.uuid4()) + ".json"
                workDir = os.path.join(baseDir, newsDomain)
                if not os.path.exists(workDir):
                    os.mkdir(workDir)

                filePathName = os.path.join(workDir, filename)

                print("filename: %s" % filePathName)
                with open(filePathName, 'w') as jf:
                    json.dump(fileData, jf, indent=2)

            except Exception:
                print("skipping article: %s" % filePathName)
                time.sleep(3)
                continue
                
        
print("done")

https://api.gdeltproject.org/api/v2/doc/doc?query=%20sourcelang:english%20domainis:nature.com&mode=artlist&maxrecords=250&STARTDATETIME=20161001000000&ENDDATETIME=20161231000000&format=json
data for nature.com 


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
import random

# base line: Generate a headline by randomly picking sequence of words for each position in the headline.
# For each position in the headline, create a dictionary and map the possible following words
# Inspired by https://www.perlmonks.org/index.pl?node_id=94856

# Headline length is fixed 10 words
fixedLength = 10
dictList = [{} for i in range(0, fixedLength)]

for news in headlines:
    words = [word for word in news.split(" ") if word and len(word) > 1]
    words = words[0:fixedLength]
          
    for index, w in enumerate(reversed(words)):
        if index == 0:
            followingWord = w
            continue
        
        cDict = dictList[index]
        if w not in cDict:
            cDict[w] = []

        cDict[w].append(followingWord)
        followingWord = w
     
rDict = reversed(dictList[1:])
newSent = []
nextValue = None
for pos in rDict:

    if not nextValue:
        nextValue = random.sample((pos.keys()), 1).pop()

    possNextList = pos[nextValue]
    newSent.append(nextValue)
    nextValue = random.sample(possNextList, 1).pop()
        
print(" ".join(newSent))


Boy Who Wasnt Immunized For Tetanus Spent 57 Days


In [3]:
#!pip install goose3
from goose3 import Goose
g = Goose()

url = urls[0]

article = g.extract(url=url)
article.title
article.cleaned_text[:150]


'Measles Is Spiking Around The Globe. How Worried Should We Be?\n\nThe world has two kinds of measles problems.\n\nIn low-income countries like Madagascar '